In [1]:
import torch as t
import torch.nn as nn
from torch.autograd import Variable as V
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.param1=nn.Parameter(t.rand(3,3))
        self.submodel1=nn.Linear(3,4)
    def forward(self,input):
        x=self.param1@input                           ####没看懂
        x=self.submodel1(x)
        return x
net=Net()
net

Net(
  (submodel1): Linear(in_features=3, out_features=4, bias=True)
)

In [2]:
import torch as t
a=t.rand(3,3)
a

tensor([[0.9654, 0.4470, 0.8164],
        [0.6350, 0.5947, 0.7899],
        [0.3980, 0.9574, 0.0808]])

In [3]:
net._modules

OrderedDict([('submodel1', Linear(in_features=3, out_features=4, bias=True))])

In [4]:
net._parameters

OrderedDict([('param1',
              Parameter containing:
              tensor([[0.4229, 0.1939, 0.5243],
                      [0.3223, 0.4883, 0.1610],
                      [0.1469, 0.6190, 0.7634]], requires_grad=True))])

In [5]:
net.param1

Parameter containing:
tensor([[0.4229, 0.1939, 0.5243],
        [0.3223, 0.4883, 0.1610],
        [0.1469, 0.6190, 0.7634]], requires_grad=True)

In [6]:
net._parameters['param1']

Parameter containing:
tensor([[0.4229, 0.1939, 0.5243],
        [0.3223, 0.4883, 0.1610],
        [0.1469, 0.6190, 0.7634]], requires_grad=True)

In [7]:
for name,param in net.named_parameters():
    print(name,param.size())

param1 torch.Size([3, 3])
submodel1.weight torch.Size([4, 3])
submodel1.bias torch.Size([4])


In [8]:
for  name,model in net.named_modules():
    print(name,model)

 Net(
  (submodel1): Linear(in_features=3, out_features=4, bias=True)
)
submodel1 Linear(in_features=3, out_features=4, bias=True)


In [9]:
bn=nn.BatchNorm1d(2)
input=V(t.rand(3,2),requires_grad=True)
print(input)
output=bn(input)
print(output)
print(bn._buffers)

tensor([[0.8097, 0.6734],
        [0.6179, 0.6596],
        [0.4878, 0.5501]], requires_grad=True)
tensor([[ 1.2947,  0.8266],
        [-0.1554,  0.5779],
        [-1.1393, -1.4046]], grad_fn=<NativeBatchNormBackward>)
OrderedDict([('running_mean', tensor([0.0638, 0.0628])), ('running_var', tensor([0.9026, 0.9005])), ('num_batches_tracked', tensor(1))])


In [10]:
input=V(t.arange(0,12,1.0).view(3,4))
print(input)
dropout=nn.Dropout(0.5,inplace=False)
output=dropout(input)
print(output)


tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]])
tensor([[ 0.,  0.,  4.,  0.],
        [ 8.,  0., 12.,  0.],
        [16., 18.,  0., 22.]])


In [11]:
dropout.training=False
output=dropout(input)
print(output)

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]])


In [12]:
net.train()
print(net.training,net.submodel1.training)
net.eval()
print(net.training,net.submodel1.training)

True True
False False


In [13]:

class VGG(nn.Module):
    def __init__(self,num_classes=1000):
        super(VGG,self).__init__()
        layers=[]
        in_dim=3
        out_dim=64
        for i in range(13):
            layers+=[nn.Conv2d(in_dim,out_dim,3,1,1),nn.ReLU(inplace=True)]
            in_dim=out_dim
            if i ==1 or i ==3 or i ==6 or i ==9 or i ==12 :
                layers+=[nn.MaxPool2d(2,2)]
                if i!=9:
                    out_dim=out_dim*2
        self.features=nn.Sequential(*layers)
        self.classifier=nn.Sequential(
            nn.Linear(512*7*7,4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096,4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),

        )
    def forward(self,x):
        x=self.features(x)
        x=x.view(x.size(0),-1)
        x = self.classifier(x)
        return x

In [14]:
model=VGG(10)
print(model)
model.features[0]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [15]:
from torchvision import models
model_vision=models.resnet18()
print(model_vision)
print(model_vision.layer1)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [16]:
#hook机制。不改变网络的定义代码，也不需要在forward函数中return某个感兴趣层的输出
features_in_hook = []
features_out_hook = []

temp1=t.Tensor()
temp2=t.Tensor()
in1=t.rand(1,3,256,256)
# print(input)

def hook(module,fea_in,fea_out):      #用户可以自行决定拦截了中间信息之后要做什么
    print("hooker working")
    features_in_hook.append(fea_in)  #原地操作
    features_out_hook.append(fea_out)  #原地操作
handle=model_vision.layer1.register_forward_hook(hook)
_ =model_vision(in1)
handle.remove()
print(features_in_hook)
print(features_out_hook)

hooker working
[(tensor([[[[2.4395, 2.4395, 0.8226,  ..., 1.3585, 1.3585, 0.3031],
          [2.4395, 2.4395, 1.6206,  ..., 1.3585, 1.3585, 1.1862],
          [2.6118, 2.6118, 1.6206,  ..., 1.3096, 2.7709, 2.7709],
          ...,
          [2.1057, 1.3046, 1.1924,  ..., 1.0106, 0.4196, 0.6698],
          [1.3739, 1.1609, 1.1924,  ..., 0.6886, 1.2327, 1.2327],
          [2.6918, 2.6918, 1.2930,  ..., 1.6630, 2.2309, 2.2309]],

         [[0.9042, 0.4002, 0.0000,  ..., 1.1256, 0.6720, 0.0000],
          [0.9042, 0.2524, 2.6517,  ..., 1.8323, 1.8323, 0.6253],
          [0.5601, 1.8687, 2.6517,  ..., 1.8323, 1.8323, 1.3493],
          ...,
          [0.7499, 0.9198, 1.1154,  ..., 1.7535, 0.5058, 0.5352],
          [0.7697, 0.9134, 1.2735,  ..., 1.7535, 1.2930, 1.2930],
          [0.4527, 1.4107, 1.2735,  ..., 0.9796, 2.1058, 2.1058]],

         [[3.3223, 3.3223, 2.1590,  ..., 1.5863, 3.2830, 3.2830],
          [3.3223, 3.3223, 2.1590,  ..., 1.5863, 3.2830, 3.2830],
          [2.7311, 2.7311

In [17]:
module=nn.Module()
module.param=nn.Parameter(t.ones(2,2))
module._parameters

OrderedDict([('param',
              Parameter containing:
              tensor([[1., 1.],
                      [1., 1.]], requires_grad=True))])

In [18]:
submodule1=nn.Linear(2,2)
submodule2=nn.Linear(2,2)
module_list=[submodule1,submodule2]
module.submodules=module_list
print(module._modules)

OrderedDict()


In [19]:
print(module.__dict__.get('submodules'))

[Linear(in_features=2, out_features=2, bias=True), Linear(in_features=2, out_features=2, bias=True)]


In [20]:
module_list=nn.ModuleList(module_list)
module.submodules=module_list
#isinstance() 函数来判断一个对象是否是一个已知的类型
print(isinstance(module_list,nn.Module))

True


In [21]:
print(module._modules)

OrderedDict([('submodules', ModuleList(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
))])


In [22]:
print(module.__dict__.get('submodules'))

None


In [23]:
getattr(module,'training')

True

In [24]:
module.attr1=2
getattr(module,'attr1')

2